<a href="https://colab.research.google.com/github/tpgjs97/company-project/blob/master/%EC%A0%84%EC%B2%98%EB%A6%AC/smartnurse_%EC%A0%84%EC%B2%98%EB%A6%AC%EC%BD%94%EB%93%9C_%EC%84%B8%ED%97%8C_(2023_11_21).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk pandas konlpy

In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 라이브러리 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import nltk
import pandas as pd
import re

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# NLTK 리소스 다운로드
nltk.download('punkt')
nltk.download('stopwords')


# 데이터 불러오기

In [617]:
# ncp JSON 파일 열기
ncp_filename = '/content/drive/MyDrive/ColabNotebooks/YearDreamSchool/FinalProject/Data/nursingrecord_ncp.json'
with open(ncp_filename, 'r') as file:
    ncp_data = json.load(file)

# aws JSON 파일 열기
aws_filename = '/content/drive/MyDrive/ColabNotebooks/YearDreamSchool/FinalProject/Data/nursingrecord_aws.json'
with open(aws_filename, 'r') as file:
    aws_data = json.load(file)

In [618]:
# JSON 문자열을 파이썬 딕셔너리로 변환하는 함수
def parse_json(json_str):
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        return {}  # JSON 파싱 에러 시 빈 딕셔너리 반환

# 각 레코드에 대해 'content' 필드를 파싱
for item in ncp_data:
    item['content'] = parse_json(item['content'])

for item in aws_data:
    item['content'] = parse_json(item['content'])


# 데이터프레임 생성
ncp_df = pd.DataFrame(ncp_data)
aws_df = pd.DataFrame(aws_data)

In [ ]:
df = pd.concat([ncp_df, aws_df])
df['content'] = df['content'].astype(str)
df = df.drop_duplicates()
df

# 전처리

In [620]:
NANDA_columns = ['domain', 'class', 'diagnosis', 'collectingData', 'goal', 'plan', 'interventions', 'evaluation']
SOAPIE_columns = ['subjective', 'objective', 'assessment', 'planning', 'interventions', 'evaluation']
FOCUS_DAR_columns = ['focus', 'data', 'action', 'response']
NARRATIVE_NOTES_columns = ['narrativeNote']
NURSING_columns = ['assessment', 'diagnosisRelate', 'diagnosis', 'goal', 'plan', 'interventions', 'evaluation']

def pre_processing(ncp_df, aws_df, record_type, columns)  :
    # 특정 'record_type' 행들만 필터링
    record_type_dict = {'NANDA' : 0, 'SOAPIE' : 1, 'FOCUS_DAR' : 2, 'NARRATIVE_NOTES' : 3, 'NURSING' : 4}
    ncp_record_type = ncp_df[ncp_df['record_type'] == record_type_dict[record_type]]
    aws_record_type = aws_df[aws_df['record_type'] == record_type_dict[record_type]]

    # 'content' 열을 확장하여 새로운 데이터 프레임 생성
    ncp_content_df = pd.json_normalize(ncp_record_type['content'])
    aws_content_df = pd.json_normalize(aws_record_type['content'])

    # 필요한 열만 가져오기
    ncp_content_df = ncp_content_df[columns]
    aws_content_df = aws_content_df[columns]

    # 두 데이터프레임 합치기
    df = pd.concat([ncp_content_df, aws_content_df])

    # 중복된 행 제거
    df = df.drop_duplicates()
    return df

# 전처리 한 데이터프레임 불러오기
nanda_df = pre_processing(ncp_df, aws_df, 'NANDA', NANDA_columns)
soapie_df = pre_processing(ncp_df, aws_df, 'SOAPIE', SOAPIE_columns)
focus_dar_df = pre_processing(ncp_df, aws_df, 'FOCUS_DAR', FOCUS_DAR_columns)
narrative_notes_df = pre_processing(ncp_df, aws_df, 'NARRATIVE_NOTES', NARRATIVE_NOTES_columns)
nursing_df = pre_processing(ncp_df, aws_df, 'NURSING', NURSING_columns)

In [621]:
# 필터링 조건에 맞지 않는 행을 확인하는 함수
def is_valid_row(text):
    # 숫자만 있는 경우
    if re.fullmatch(r'\d+', text):
        return False
    # 영어 알파벳만 있는 경우
    if re.fullmatch(r'[a-zA-Z]+', text):
        return False
    # 한글 자음 또는 모음만 있는 경우
    if re.fullmatch(r'[ㄱ-ㅎㅏ-ㅣ]+', text):
        return False
    # 특수 문자만 있는 경우
    if re.fullmatch(r'[^\w\s]+', text):
        return False

    return True

# 필터링 조건에 맞지 않는 단어를 제거하는 함수
def remove_invalid_words(text):
    if is_valid_row(str(text)) :
        valid_words = text
    else :
        valid_words = pd.NA # 빈 문자열 대신 pd.NA 반환

    return valid_words

# 조건에 맞는 단어로 대체하는 함수
def text_preprocessing(text):
    if text is not pd.NA :
        text = text.replace('\\n', ' ')
        text = text.replace('\n', ' ')
        txt = re.sub('[^가-힣a-zA-Z0-9./%]', ' ', text) # 한글과 영어 소문자만 남기고 다른 글자 모두 제거
    else :
        txt = pd.NA

    return txt

# 글자 수가 특정 개수를 넘어가지 못한 행을 제거하는 함수
def filter_rows_by_length(df, min_length=2):
    condition = df.apply(lambda col: col.apply(lambda x: len(str(x))) >= min_length).all(axis=1)
    df_filtered = df[condition]

    return df_filtered

In [622]:
def function_apply(df) :
    # 조건을 만족하는 단어만 남김
    for column in df.columns :
        df[column] = df[column].apply(remove_invalid_words)
        df[column] = df[column].apply(text_preprocessing)

    # Null값이 있는 행을 모두 제거
    df = df.dropna()

    # 글자 수를 필터로한 함수 적용 (글자수가 3 미만인 행)
    df = filter_rows_by_length(df, 3)

    return df

In [623]:
nanda_df = function_apply(nanda_df)
soapie_df = function_apply(soapie_df)
focus_dar_df = function_apply(focus_dar_df)
narrative_notes_df = function_apply(narrative_notes_df)
nursing_df = function_apply(nursing_df)

In [624]:
#pd.set_option('display.max_rows',10)
print(len(nanda_df), len(soapie_df), len(focus_dar_df), len(narrative_notes_df), len(nursing_df))
print(len(nanda_df) + len(soapie_df) + len(focus_dar_df) + len(narrative_notes_df) + len(nursing_df))

1342 713 1659 13336 299
17349


## Discussion

### 1. Nanda의 collectingData에는 주관적 정보(S) 객관적 정보(O)가 구분되어있는 경우가 약 8% 정도 존재.


In [ ]:
print(f"주관적, 객관적 정보로 나누어져 기술된 정보의 개수 : {len(nanda_df[(nanda_df['collectingData'].str.startswith('S ')) | (nanda_df['collectingData'].str.startswith('주관적 '))])} \n전체 개수 : {len(nanda_df)}", '\n')

for i in nanda_df[(nanda_df['collectingData'].str.startswith('S ')) | (nanda_df['collectingData'].str.startswith('주관적 '))]['collectingData'].index :
    print(nanda_df[(nanda_df['collectingData'].str.startswith('S ')) | (nanda_df['collectingData'].str.startswith('주관적 '))]['collectingData'][i])